# Data Acquisition

### Import Libraries


In [1]:
import ee
import geemap
import os
import pandas as pd
import rasterio
import matplotlib.pyplot as plt


current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
work_dir = os.path.join(project_root, 'data', 'raw')
os.makedirs(work_dir, exist_ok=True)

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/ozantuncbilek/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/ozantuncbilek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
ee.Authenticate()
ee.Initialize(project='manifest-pride-258211')

### Temporal Selection: Identification of Peak Heat Events

To focus the analysis on maximum urban heat stress, we utilize daily MODIS LST data (MOD11A1) to identify the **top 3 hottest cloud-free summer days** for each year. These specific dates serve as the temporal targets for the subsequent retrieval of high-resolution Landsat and Sentinel imagery, ensuring we analyze the city under its most critical thermal conditions.

In [3]:
def get_modis_top_n_hottest_days(start_year=2014, end_year=2024, n_days=3):
    """
    Finds the top N hottest cloud-free summer days for Hamburg using MODIS LST data.
    Returns a DataFrame where each year maps to a list of dictionaries.
    """
    hamburg = ee.Geometry.Point(9.9937, 53.5511).buffer(5000)
    results = {}

    for year in range(start_year, end_year + 1):
        try:
            modis_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
                .filterBounds(hamburg) \
                .filterDate(f'{year}-05-15', f'{year}-09-15')

            def compute_lst(img):
                mean_lst = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=hamburg, scale=1000).get('LST_Day_1km')
                return ee.Feature(None, {'lst': mean_lst, 'date': img.date().format('YYYY-MM-dd')})

            lst_features = modis_collection.map(compute_lst).filter(ee.Filter.notNull(['lst']))

            if lst_features.size().getInfo() == 0:
                continue

            # Instead of .first(), use .limit(n) to get the top N days
            hottest_list = lst_features.sort('lst', False).limit(n_days).getInfo()['features']

            top_days = []
            for feature in hottest_list:
                props = feature['properties']
                if props['lst'] is not None:
                    celsius = props['lst'] * 0.02 - 273.15
                    top_days.append({
                        'date': props['date'],
                        'lst_celsius': round(celsius, 2)
                    })

            results[year] = {'top_days': top_days}
            print(f"Found top {len(top_days)} days for {year}.")

        except Exception as e:
            print(f"Error processing {year}: {str(e)}")
            continue

    return pd.DataFrame.from_dict(results, orient='index')

print("Extracting MODIS data...")
df_top_days = get_modis_top_n_hottest_days()

Extracting MODIS data...
Found top 3 days for 2014.
Found top 3 days for 2015.
Found top 3 days for 2016.
Found top 3 days for 2017.
Found top 3 days for 2018.
Found top 3 days for 2019.
Found top 3 days for 2020.
Found top 3 days for 2021.
Found top 3 days for 2022.
Found top 3 days for 2023.
Found top 3 days for 2024.


In [4]:
for year, row in df_top_days.iterrows():
    print(f"--- Year: {year} ---")

    # Get the list of candidate days for the current year
    top_days_list = row['top_days']

    # Check if the list is not empty
    if not top_days_list:
        print("  No candidate days found.")
        continue

    # Print each candidate day
    for i, day_info in enumerate(top_days_list):
        date = day_info['date']
        temp = day_info['lst_celsius']
        print(f"  {i+1}. Hottest: Date: {date}, Temp: {temp}°C")
    print("-" * 20)

--- Year: 2014 ---
  1. Hottest: Date: 2014-07-04, Temp: 34.83°C
  2. Hottest: Date: 2014-07-19, Temp: 34.75°C
  3. Hottest: Date: 2014-07-20, Temp: 34.27°C
--------------------
--- Year: 2015 ---
  1. Hottest: Date: 2015-07-05, Temp: 38.17°C
  2. Hottest: Date: 2015-07-02, Temp: 34.78°C
  3. Hottest: Date: 2015-06-12, Temp: 34.33°C
--------------------
--- Year: 2016 ---
  1. Hottest: Date: 2016-06-05, Temp: 35.11°C
  2. Hottest: Date: 2016-06-23, Temp: 34.02°C
  3. Hottest: Date: 2016-07-20, Temp: 32.88°C
--------------------
--- Year: 2017 ---
  1. Hottest: Date: 2017-05-27, Temp: 32.62°C
  2. Hottest: Date: 2017-06-02, Temp: 31.26°C
  3. Hottest: Date: 2017-07-09, Temp: 30.79°C
--------------------
--- Year: 2018 ---
  1. Hottest: Date: 2018-07-27, Temp: 37.34°C
  2. Hottest: Date: 2018-07-26, Temp: 37.22°C
  3. Hottest: Date: 2018-08-03, Temp: 37.13°C
--------------------
--- Year: 2019 ---
  1. Hottest: Date: 2019-06-30, Temp: 39.26°C
  2. Hottest: Date: 2019-07-23, Temp: 34.56°C

### Robust Landsat Data Retrieval Pipeline

This section defines the core logic for retrieving high-quality Landsat 8 thermal imagery. To address common issues like **cloud contamination** and **satellite swath gaps (black stripes)**, we implement a multi-stage filtering algorithm:

1.  **Iterative Search (Strategy A):** The function searches for images within a ±15 day window of the target date. It prioritizes images with minimal cloud cover (<20%).
2.  **Fallback Mechanism (Strategy B):** If no high-quality image is found, the criteria are relaxed to find the best available usable image, preventing data gaps in the time series.
3.  **Coverage Calculation:** A helper function calculates the percentage of valid pixels in the ROI to quantitatively ensure the image is not corrupted by masking artifacts.
4.  **LST Computation:** Converts the thermal band from Kelvin to Celsius.

In [5]:
# Hamburg coordinates
hamburg = ee.Geometry.Rectangle([9.7, 53.4, 10.3, 53.7])

def get_robust_landsat_data(target_date_str, max_cloud=20, search_radius_days=15):
    """
    Finds a cloud-free Landsat 8 image for a specific target date.
    Returns the image, the found date, and a status ('SUCCESS', 'FALLBACK', 'FAILURE').
    """
    target_date = ee.Date(target_date_str)
    image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(hamburg)

    # --- Strategy A: Iterative search for a low-cloud image ---
    for day_offset in range(search_radius_days + 1):
        start_date = target_date.advance(-day_offset, 'day')
        end_date = target_date.advance(day_offset, 'day').advance(1, 'day')

        landsat_collection = image_collection \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUD_COVER', max_cloud)) \
            .sort('CLOUD_COVER')

        if landsat_collection.size().getInfo() > 0:
            found_date_ee = ee.Date(landsat_collection.first().get('system:time_start'))
            found_date_str = found_date_ee.format('YYYY-MM-dd').getInfo()
            image = landsat_collection.mosaic().clip(hamburg)

            def mask_clouds(img):
                qa = img.select('QA_PIXEL')
                cloud_bit_mask = 1 << 3; cloud_shadow_bit_mask = 1 << 4
                mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
                return img.updateMask(mask)

            return mask_clouds(image), found_date_str, 'SUCCESS'

    # --- Strategy B: Fallback (Best available) ---
    start_date = target_date.advance(-search_radius_days, 'day')
    end_date = target_date.advance(search_radius_days, 'day').advance(1, 'day')
    fallback_collection = image_collection.filterDate(start_date, end_date).sort('CLOUD_COVER')

    if fallback_collection.size().getInfo() > 0:
        best_image = fallback_collection.first()
        if best_image.get('CLOUD_COVER').getInfo() > 60:
             return None, None, 'FAILURE'

        found_date_ee = ee.Date(best_image.get('system:time_start'))
        found_date_str = found_date_ee.format('YYYY-MM-dd').getInfo()
        image = fallback_collection.mosaic().clip(hamburg)

        def mask_clouds(img):
            qa = img.select('QA_PIXEL')
            cloud_bit_mask = 1 << 3; cloud_shadow_bit_mask = 1 << 4
            mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
            return img.updateMask(mask)

        return mask_clouds(image), found_date_str, 'FALLBACK'

    return None, None, 'FAILURE'


def get_summer_median_composite(year, geometry):
    """
    EMERGENCY FALLBACK: Creates a gap-free median composite for a specific year.
    Used ONLY when single-date images have huge holes (swath gaps like 2017).
    """
    start = f'{year}-06-01'
    end = f'{year}-08-31'
    
    collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(geometry) \
        .filterDate(start, end) \
        .filter(ee.Filter.lt('CLOUD_COVER', 40)) # Allow slightly more clouds for median input
    
    def mask_and_prep(img):
        qa = img.select('QA_PIXEL')
        # Mask Clouds and Shadows
        mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
        return img.updateMask(mask)

    # Take the MEDIAN of the whole summer (fills gaps)
    composite = collection.map(mask_and_prep).median().clip(geometry)
    return composite


def get_coverage_percentage(image, geometry):
    """Calculates valid pixel percentage."""
    valid_pixels = image.select(0).unmask(0).gt(0)
    coverage_stats = valid_pixels.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=30, maxPixels=1e9
    )
    return ee.Number(coverage_stats.values().get(0)).multiply(100)


def calculate_lst(image):
    if image is None: return None
    # Scale factors: 0.00341802 * DN + 149.0 - 273.15
    lst = image.expression(
        '(TIRS1 * 0.00341802 + 149.0) - 273.15',
        {'TIRS1': image.select('ST_B10')}
    ).rename('LST')
    return image.addBands(lst)

#### Candidate Evaluation & Image Selection

Here, we execute the selection tournament. For each year, we evaluate the **top 3 hottest candidate dates** identified in the previous step (via MODIS).

**Selection Logic:**
* The algorithm retrieves Landsat images for all candidate dates.
* It calculates the **Valid Pixel Coverage (%)** for each retrieved image.
* **Decision Rule:** The image with the highest spatial coverage (least missing data due to clouds or sensor errors) is selected as the representative LST map for that year.
* This dynamic selection ensures we avoid specific days where satellite artifacts (e.g., 2017 swath gaps) might ruin the analysis.

In [6]:
years = df_top_days.index.tolist()
lst_images = {}
final_dates = {}

# Process each year
for year in years:
    print(f"\n--- Processing Year: {year} ---")

    top_days_for_year = df_top_days.loc[year, 'top_days']
    successful_candidates = []

    # 1. Try to find the HOTTEST DAY image
    for i, day_info in enumerate(top_days_for_year):
        target_date = day_info['date']
        landsat_image, found_date, status = get_robust_landsat_data(target_date)

        if status == 'SUCCESS':
            coverage = get_coverage_percentage(landsat_image, hamburg).getInfo()
            print(f"  > Candidate {found_date}: Coverage {coverage:.2f}%")
            
            successful_candidates.append({
                'image': landsat_image,
                'found_date': found_date,
                'target_date': target_date,
                'coverage': coverage,
                'type': 'Single Date'
            })

    # 2. Decide Strategy
    selected_data = None
    
    if successful_candidates:
        # Pick best single day
        best_candidate = sorted(successful_candidates, key=lambda x: x['coverage'], reverse=True)[0]
        
        # QUALITY CHECK: Is the best single day actually broken? (e.g. 2017 with 63%)
        if best_candidate['coverage'] < 95.0:
            print(f"Best single day for {year} has low coverage ({best_candidate['coverage']:.2f}%). Switching to MEDIAN COMPOSITE.")
            
            # Use Fallback: Summer Median
            median_image = get_summer_median_composite(year, hamburg)
            median_coverage = get_coverage_percentage(median_image, hamburg).getInfo()
            
            selected_data = {
                'image': median_image,
                'found_date': 'Summer Median',
                'target_date': 'Summer Median',
                'coverage': median_coverage,
                'type': 'Median Composite'
            }
        else:
            print(f"Best single day for {year} is good ({best_candidate['coverage']:.2f}%). Keeping Hottest Day.")
            selected_data = best_candidate

    else:
        # No single days found at all, use Median
        print(f"No single days found for {year}. Using MEDIAN COMPOSITE.")
        median_image = get_summer_median_composite(year, hamburg)
        median_coverage = get_coverage_percentage(median_image, hamburg).getInfo()
        
        selected_data = {
            'image': median_image,
            'found_date': 'Summer Median',
            'target_date': 'Summer Median',
            'coverage': median_coverage,
            'type': 'Median Composite'
        }

    # 3. Final Calculation & Storage
    print(f"FINAL DECISION for {year}: Type={selected_data['type']} | Coverage={selected_data['coverage']:.2f}%")
    
    lst_images[year] = calculate_lst(selected_data['image'])
    final_dates[year] = {
        'target': selected_data['target_date'],
        'found': selected_data['found_date'],
        'coverage': selected_data['coverage']
    }


--- Processing Year: 2014 ---
  > Candidate 2014-07-10: Coverage 99.82%
  > Candidate 2014-07-10: Coverage 99.82%
  > Candidate 2014-07-10: Coverage 99.82%
Best single day for 2014 is good (99.82%). Keeping Hottest Day.
FINAL DECISION for 2014: Type=Single Date | Coverage=99.82%

--- Processing Year: 2015 ---
  > Candidate 2015-06-11: Coverage 99.73%
Best single day for 2015 is good (99.73%). Keeping Hottest Day.
FINAL DECISION for 2015: Type=Single Date | Coverage=99.73%

--- Processing Year: 2016 ---
  > Candidate 2016-06-04: Coverage 99.90%
Best single day for 2016 is good (99.90%). Keeping Hottest Day.
FINAL DECISION for 2016: Type=Single Date | Coverage=99.90%

--- Processing Year: 2017 ---
  > Candidate 2017-05-22: Coverage 31.03%
  > Candidate 2017-05-22: Coverage 31.03%
  > Candidate 2017-07-09: Coverage 63.94%
Best single day for 2017 has low coverage (63.94%). Switching to MEDIAN COMPOSITE.
FINAL DECISION for 2017: Type=Median Composite | Coverage=99.99%

--- Processing Year

#### Visual Verification of LST Data

Before proceeding to analysis, we visualize the retrieved Land Surface Temperature (LST) maps. This step serves as a **sanity check** to ensure:
1.  **Spatial Completeness:** Verifying that 2017 (gap-filled) and 2024 (cloud-corrected) cover the entire city without artifacts.
2.  **Temperature Consistency:** Confirming that urban areas appear hotter (red) and water bodies/vegetation appear cooler (blue).

In [7]:
# 1. Initialize Map (Re-initialize to clear memory and avoid double layers)
Map = geemap.Map()
Map.centerObject(hamburg, 11)

# 2. Visualization Parameters
vis_params = {
    'min': 20,
    'max': 45,
    'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000']
}

print("Adding LST layers to the map...")

# 3. Add Layers (Sorted by Year)
sorted_years = sorted(lst_images.keys())

for year in sorted_years:
    lst_image = lst_images[year]
    
    # Extract metadata for layer naming
    info = final_dates[year]
    is_median = "Median" in str(info.get('target', ''))
    type_label = " (Median Composite)" if is_median else ""
    
    layer_name = f"{year} LST{type_label}"
    
    # Show ALL years by default (as requested)
    # Note: Enabling all layers might impact browser performance
    #show_layer = True
    show_layer = (year == 2024)  # Example: Only show 2024 by default
    
    # Add ONLY the 'LST' band to avoid palette visualization errors
    Map.addLayer(lst_image.select('LST'), vis_params, layer_name, shown=show_layer)

# 4. Add Colorbar and Controls
try:
    Map.add_colorbar(vis_params, label="Land Surface Temperature (Celsius)")
except Exception as e:
    print(f"Colorbar warning: {e}")

Map.addLayerControl()

# Display Map
Map

Adding LST layers to the map...


Map(center=[53.55019811195351, 10.000000000000183], controls=(WidgetControl(options=['position', 'transparent_…

### Optical Data Acquisition: Sentinel-2 & Landsat-8 Harmonization

To calculate spectral indices like **NDVI** (Vegetation) and **NDBI** (Built-up Area), we require optical satellite data (Red, NIR, SWIR bands).

**Challenge:** The Sentinel-2 mission (10m resolution) only provides data from mid-2015 onwards. The years 2014 and 2015 are uncovered.

**Strategy:** We implement a **Harmonization Pipeline**:
1.  **2016-2024 (Sentinel-2):** We retrieve high-resolution (10m) Surface Reflectance data (`COPERNICUS/S2_SR_HARMONIZED`).
2.  **2014-2015 (Landsat-8 Fallback):** We utilize Landsat-8 (30m) data. Crucially, we **rename the bands** to match Sentinel-2 conventions (`Red`, `NIR`, `SWIR`). During the export/analysis phase, these images will be implicitly upscaled to 10m to match the spatial grid of the later years.

**Preprocessing:**
* Both datasets undergo rigorous **Cloud Masking** using their respective Quality Assessment (QA) bands.
* A **Median Composite** is generated for the summer months (June-August) to ensure a cloud-free, phenologically consistent representation of the city.

In [10]:
# Hamburg coordinates
hamburg = ee.Geometry.Rectangle([9.7, 53.4, 10.3, 53.7])

# --- 1. LANDSAT 8 FUNCTIONS ---
def mask_landsat8_clouds(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask)

def create_landsat_summer_composite(year):
    print(f"\n--- Creating LANDSAT-8 composite (Fallback) for Summer {year} ---")
    start_date = f'{year}-06-01'
    end_date = f'{year}-08-31'

    collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(hamburg) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUD_COVER', 30)) \
        .map(mask_landsat8_clouds)
    
    return collection.median().clip(hamburg).select(['SR_B4', 'SR_B5', 'SR_B6'], ['Red', 'NIR', 'SWIR'])

# --- 2. SENTINEL-2 FUNCTIONS (Homogeneity & Speed Fix) ---

def mask_s2_sr(image):
    scl = image.select('SCL')
    mask = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(7))
    return image.updateMask(mask)

def mask_s2_l1c(image):
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(mask)

def create_sentinel2_summer_composite(year):
    if year < 2016: return None

    print(f"\n--- Creating SENTINEL-2 composite for Summer {year} ---")
    start_date = f'{year}-06-01'
    end_date = f'{year}-08-31'

    # 1. Collection & Function Selection
    if year >= 2017:
        collection_id = "COPERNICUS/S2_SR_HARMONIZED"
        mask_func = mask_s2_sr
        # FIX: Pre-select only necessary bands to avoid "Incompatible bands" error
        bands_to_select = ['B4', 'B8', 'B11', 'SCL'] 
    else:
        print(f"  > Using Level-1C collection for {year}")
        collection_id = "COPERNICUS/S2"
        mask_func = mask_s2_l1c
        bands_to_select = ['B4', 'B8', 'B11', 'QA60']

    # 2. Dynamic Cloud Filter
    cloud_limit = 80 if year == 2017 else 30
    if year == 2017: print(f"  > Applying relaxed cloud filter ({cloud_limit}%) for gap filling.")

    # 3. Build Collection
    collection = ee.ImageCollection(collection_id) \
        .filterBounds(hamburg) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_limit)) \
        .select(bands_to_select) \
        .map(mask_func) # Now map the mask over the cleaner collection

    composite = collection.median().clip(hamburg)

    return composite.select(['B4', 'B8', 'B11'], ['Red', 'NIR', 'SWIR'])

def get_valid_pixel_percentage(image, geometry):
    """Calculates the percentage of valid (unmasked) pixels within a geometry."""
    # Create a mask where valid pixels are 1, masked pixels are 0
    mask = image.select(0).mask()
    
    # Calculate the mean of the mask (which equals the percentage of valid pixels)
    stats = mask.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=100, # Fast approximation scale
        maxPixels=1e9
    )
    # Return percentage (0-100)
    return stats.values().get(0).getInfo() * 100

#### Optical Data Processing Loop

This cell iterates through the study period (2014-2024). It applies the specific selection logic:

Years 2014-2015: Generate Landsat-8 Fallback Composites (30m, harmonized).

Years 2016-2024: Generate Sentinel-2 High-Resolution Composites (10m).

It also performs a validity check to ensure the generated images contain valid data before storing them in the dictionary.

In [11]:
composite_images = {}
years_to_process = range(2014, 2025)

print("\n--- Starting Processing with Coverage Checks ---")

for year in years_to_process:
    
    # 1. Create Composite based on year logic
    if year < 2016: 
        composite = create_landsat_summer_composite(year)
    else: 
        composite = create_sentinel2_summer_composite(year)

    if composite:
        try:
            # 2. Calculate Coverage
            coverage = get_valid_pixel_percentage(composite, hamburg)
            
            # Print status
            if coverage < 98.0:
                print(f"{year}: Low Coverage! Only {coverage:.2f}% valid pixels (Gaps detected).")
            else:
                print(f"{year}: Good Coverage ({coverage:.2f}%).")
            
            # 3. Store the image
            composite_images[year] = composite
            
        except Exception as e:
            print(f"  > Error processing {year}: {e}")

print("\nProcessing complete.")


--- Starting Processing with Coverage Checks ---

--- Creating LANDSAT-8 composite (Fallback) for Summer 2014 ---
2014: Good Coverage (99.81%).

--- Creating LANDSAT-8 composite (Fallback) for Summer 2015 ---
2015: Good Coverage (99.79%).

--- Creating SENTINEL-2 composite for Summer 2016 ---
  > Using Level-1C collection for 2016
2016: Good Coverage (99.85%).

--- Creating SENTINEL-2 composite for Summer 2017 ---
  > Applying relaxed cloud filter (80%) for gap filling.
2017: Good Coverage (98.37%).

--- Creating SENTINEL-2 composite for Summer 2018 ---
2018: Good Coverage (99.30%).

--- Creating SENTINEL-2 composite for Summer 2019 ---
2019: Good Coverage (99.40%).

--- Creating SENTINEL-2 composite for Summer 2020 ---
2020: Good Coverage (99.56%).

--- Creating SENTINEL-2 composite for Summer 2021 ---
2021: Good Coverage (99.60%).

--- Creating SENTINEL-2 composite for Summer 2022 ---
2022: Good Coverage (99.85%).

--- Creating SENTINEL-2 composite for Summer 2023 ---
2023: Good Cov

#### Visual Verification of Optical Data (False Color Composite)

We generate a visualization for **all processed years (2014-2024)** using a False Color Composite.

**Why False Color?**
We specifically retrieved **Red, NIR, and SWIR** bands to calculate spectral indices (NDVI/NDBI) efficiently. Since we did not download Blue or Green bands (to optimize processing and storage), a True Color (RGB) visualization is not possible with the current dataset.

**Visualization Key:**
* **SWIR (Red Channel):** Highlights urbanization and impervious surfaces (appears purple/brown).
* **NIR (Green Channel):** Highlights vegetation health (appears vibrant green).
* **Red (Blue Channel):** Adds contrast.

In [12]:
import geemap

# 1. Initialize Map
# Center explicitly on Hamburg to ensure correct zoom level
Map = geemap.Map()
Map.centerObject(hamburg, 11)

# 2. Visualization Parameters
# We define different parameters because Landsat-8 and Sentinel-2 have different data scales.

# Sentinel-2 (Values typically 0-10000)
vis_params_s2 = {
    'bands': ['SWIR', 'NIR', 'Red'],
    'min': 0,
    'max': 3500,
    'gamma': 1.4
}

# Landsat-8 (Values typically higher unscaled integers)
vis_params_l8 = {
    'bands': ['SWIR', 'NIR', 'Red'],
    'min': 7000,
    'max': 25000,
    'gamma': 1.4
}

print("Adding All Optical Layers (2014-2024)...")

sorted_years = sorted(composite_images.keys())

for year in sorted_years:
    image = composite_images[year]
    
    # 3. Select appropriate parameters and label based on satellite source
    if year < 2016:
        # Landsat-8 (2014-2015)
        params = vis_params_l8
        layer_name = f"{year} Optical (Landsat-8)"
    else:
        # Sentinel-2 (2016-2024)
        params = vis_params_s2
        
        # Add suffix to distinguish the special 2016 fix (L1C) vs Standard (SR)
        suffix = "L1C" if year == 2016 else "SR"
        layer_name = f"{year} Optical (Sentinel-2 {suffix})"

    # 4. Layer Visibility Logic
    is_shown = True
    #is_shown = (year == 2024)  # Example: Only show 2024 by default

    Map.addLayer(image, params, layer_name, shown=is_shown)

# Add Layer Control to the map
Map.addLayerControl()

print("Map Ready. All years are available in the Layer Control panel.")
Map

Adding All Optical Layers (2014-2024)...
Map Ready. All years are available in the Layer Control panel.


Map(center=[53.55019811195351, 10.000000000000183], controls=(WidgetControl(options=['position', 'transparent_…

### Data Export: Land Surface Temperature (LST)

We export the processed LST images to the local drive as GeoTIFF files.
To preserve previous experiments, we save these outputs to a new versioned directory: `data/raw_v2/landsat_lst`.

* **Format:** GeoTIFF (.tif)
* **Resolution:** 30m (Standard Landsat Thermal resolution)
* **Band:** 'LST' (Celsius)

In [ ]:
# 1. Setup Output Directory (VERSION 2)
# We use 'raw_v2' to avoid overwriting the old dataset
project_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'notebooks' else os.getcwd()
output_dir = os.path.join(project_root, 'data', 'raw_v2', 'landsat_lst')

# Create directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

print(f"Target Directory: {output_dir}")

# 2. Download Loop
if 'lst_images' in globals():
    for year, lst_image in lst_images.items():
        
        # Define filename
        filename = f"LST_{year}_Hamburg.tif"
        full_path = os.path.join(output_dir, filename)
        
        # Check if file already exists
        if os.path.exists(full_path):
            print(f"Skipping {year}, file already exists.")
            continue

        print(f"Downloading {year} (LST)...")

        try:
            # Export the image locally
            geemap.ee_export_image(
                lst_image.select('LST'),     # Select only the LST band
                filename=full_path,
                scale=30,                    # 30m resolution for Landsat LST
                region=hamburg,              # Region of interest
                file_per_band=False
            )
            print(f"Successfully saved: {filename}")
            
        except Exception as e:
            print(f"Error downloading {year}: {e}")

print("LST Export tasks completed!")

### Data Export: Optical Composites (Sentinel-2 & Landsat-8)

We export the optical data (Red, NIR, SWIR bands) used for calculating spectral indices.
These are saved to `data/raw_v2/sentinel_2`.

* **Resolution:** All images are exported at **10m resolution**.
    * *Sentinel-2:* Native 10m.
    * *Landsat-8:* Upscaled from 30m to 10m during export to match the Sentinel grid for the model.
* **Bands:** Red, NIR, SWIR.

In [ ]:
import os
import geemap
import ee

# 1. Setup Output Directory (VERSION 2)
project_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'notebooks' else os.getcwd()
output_dir = os.path.join(project_root, 'data', 'raw_v2', 'sentinel_2')
os.makedirs(output_dir, exist_ok=True)

print(f"Target Directory: {output_dir}")

# 2. Download Loop
if 'composite_images' in globals():
    for year, composite_image in composite_images.items():
        
        # File name format: Hamburg_Composite_2024.tif
        filename = f"Hamburg_Composite_{year}.tif"
        full_path = os.path.join(output_dir, filename)
        
        # Check existence
        if os.path.exists(full_path):
            print(f"Skipping {year}, file already exists.")
            continue

        try:
            # Determine satellite source for logging
            sat_name = "Landsat-8" if year < 2016 else "Sentinel-2"

            # --- BAND SELECTION ---
            # Select only Red, NIR, SWIR
            selected_image = composite_image.select(['Red', 'NIR', 'SWIR'])

            # FORCE 10m SCALE
            current_scale = 10 

            print(f"Downloading {year} ({sat_name}) as {filename}...")
            print("  -> Splitting into tiles (this may take a moment)...")
            
            # USE download_ee_image (Tiled Download) INSTEAD OF ee_export_image
            geemap.download_ee_image(
                image=selected_image,
                filename=full_path,
                region=hamburg,
                scale=current_scale,
                crs='EPSG:4326' # Standard Lat/Lon coordinate system
            )
            print(f"Successfully saved {year}")

        except Exception as e:
            print(f"Error downloading {year}: {e}")

print("Optical Export completed.")

### Summary and Next Steps

In this notebook, **data acquisition** was prepared to determine the **hottest day** and to download satellite imagery from **Landsat 8 for LST** and **Sentinel-2**.  
These datasets were saved in the `data/raw/` directory.

In the next notebook, **`03_data_processing.ipynb`**, we will combine *LST* with **spectral indices** (e.g., NDVI) to create a **multi-channel tensor**, which will serve as the final input for the **U-Net model**.
